<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/experiments/SimCLR_1_DR_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth, drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

import argparse
import torch
import torch.backends.cudnn as cudnn

import gdown

In [ ]:
# Mount Drive
drive.mount('/content/drive')
auth.authenticate_user()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def download(data, url):
    # Download dataset
    import zipfile
    url = url
    output = "{}.zip".format(data)
    gdown.download(url, output, quiet=False)

    # Uncompress dataset
    local_zip = '{}.zip'.format(data)
    zip_ref = zipfile.ZipFile(local_zip, "r")
    zip_ref.extractall()
    zip_ref.close()

In [ ]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [ ]:
UNLABELED = 'train_voets'

In [ ]:
URL_UNLABELED = data_samples[UNLABELED]
download(UNLABELED, URL_UNLABELED)

Downloading...
From: https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO
To: /content/sample@20000.zip
1.08GB [00:07, 138MB/s] 


In [ ]:
!git clone -b develop https://github.com/jmarrietar/SimCLR-1.git

Cloning into 'SimCLR-1'...
remote: Enumerating objects: 617, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 617 (delta 47), reused 56 (delta 27), pack-reused 538
Receiving objects: 100% (617/617), 82.45 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (350/350), done.


In [ ]:
cd SimCLR-1/

/content/SimCLR-1


In [ ]:
!python run.py -data ./datasets -dataset-name dr --learning-rate 0.00001 --log-every-n-steps 100 --epochs 100 --unlabeled ../$UNLABELED --batch-size 64 --arch resnet50 

Device is cuda
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100% 97.8M/97.8M [00:00<00:00, 135MB/s]
Resuming Training ...
2021-05-17 04:22:11.661730: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Start SimCLR training for 200 epochs.
Training with gpu: False.
  0% 0/312 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This Dat

In [ ]:
RESUME_EPOCHS = 5

resume_model_name = "checkpoint_{:04d}.pth.tar".format(RESUME_EPOCHS)
checkpoint = torch.load(
    "/content/drive/MyDrive/Colab Notebooks/SimCLR/models/SimCLR-1-DR-pytorch/{}".format(
        resume_model_name
    ),
    map_location='cuda',
)

print("Optimizer info")
print('lr: {}'.format(checkpoint["optimizer"]["param_groups"][0]["lr"]))
print('initial_lr: {}'.format(checkpoint["optimizer"]["param_groups"][0]["initial_lr"]))

Optimizer info
lr: 3e-05
initial_lr: 3e-05


In [ ]:
#!python run.py -data ./datasets -dataset-name dr --learning-rate 0.00003 --log-every-n-steps 100 --epochs 10 --resume-epochs $RESUME_EPOCHS --unlabeled ../$UNLABELED --batch-size 64 --arch resnet50 